In [1]:
# Author: Michael Munje

In [2]:
import sqlite3
conn = sqlite3.connect('final.db')
cur = conn.cursor()

In [3]:
def fetch_table_names(cur: sqlite3.Cursor) -> [str]:
    '''
    Retrieves all tables in the database
    '''
    
    cur.execute('SELECT name FROM sqlite_master WHERE type = \'table\';')
    return [x[0] for x in cur.fetchall()]

def fetch_column_names(cur: sqlite3.Cursor, table: str) -> [str]:
    '''
    Retrieves all columns in a table
    '''
    
    cur.execute('PRAGMA table_info(' + table + ');')
    return [x[1] for x in cur.fetchall()]

def fetch_col_values(cur: sqlite3.Cursor, table: str, col: str) -> []:
    '''
    Returns all values for a specific column
    '''
    
    cur.execute('SELECT ' + col + ' FROM ' + table + ';')
    return [x[0] for x in cur.fetchall()]

In [4]:
# Review:
# Database contains tables
# You can think of these tables as a database version of a dataframe
# Therefore, tables contain columns and rows (each entry)

In [5]:
# Retrieves all possible tables in the database
fetch_table_names(cur)

['SCHOOL_TRAIN', 'SCHOOL_TEST', 'POKEMON_TRAIN', 'POKEMON_TEST']

In [6]:
# Retrieves all possible columns in a table
fetch_column_names(cur, 'POKEMON_TRAIN')

['rowid',
 'unique_id',
 'types',
 'abilities',
 'base_happiness',
 'height_m',
 'weight_kg',
 'poke_stats',
 'is_legendary']

In [7]:
fetch_column_names(cur, 'POKEMON_TEST')

['rowid',
 'unique_id',
 'types',
 'abilities',
 'base_happiness',
 'height_m',
 'weight_kg',
 'poke_stats']

In [8]:
# Retrieves all values in a column
fetch_col_values(cur, 'POKEMON_TRAIN', 'poke_stats')[:1]

['nan, 80.0, 55.0, 25.0, 35.0, 35.0']

In [9]:
# Note: You can fetch all the values at once with SELECT * FROM TABLE_NAME;
# but this is NOT a good idea if you have a ton of data
# And since I have 1500 columns, I will not do such

In [10]:
import pandas as pd
train = pd.DataFrame()

# You now know how to easily query (fetch) column data from a database
# And also how to find all possible column names
# How can you use this knowledge to construct a dataframe?
# Good luck

In [11]:
train = pd.read_sql_query(
'''select
rowid,
unique_id,
types,
abilities,
base_happiness,
height_m,
weight_kg,
poke_stats,
is_legendary
from POKEMON_TRAIN''', conn)
test = pd.read_sql_query(
'''select
rowid,
unique_id,
types,
abilities,
base_happiness,
height_m,
weight_kg,
poke_stats
from POKEMON_TEST''', conn)

In [12]:
train.head()

,rowid,unique_id,types,abilities,base_happiness,height_m,weight_kg,poke_stats,is_legendary
0,1,665,"fighting, nan","['Guts', 'Sheer Force', 'Iron Fist']",70.0,0.6,12.5,"nan, 80.0, 55.0, 25.0, 35.0, 35.0",0
1,2,288,"dragon, nan","['Bulletproof', 'Soundproof', 'Overcoat']",70.0,0.6,29.7,"nan, nan, 65.0, 45.0, 45.0, 45.0",0
2,3,349,"poison, fire","['Corrosion', 'Oblivious']",70.0,1.2,22.2,"nan, 64.0, 60.0, 111.0, 60.0, 117.0",0
3,4,260,"ground, nan","['Rock Head', 'Lightningrod', 'Battle Armor']",70.0,0.4,6.5,"nan, 50.0, 95.0, 40.0, 50.0, 35.0",0
4,5,536,"water, nan","['Torrent', 'Sheer Force']",70.0,0.6,9.5,"nan, 65.0, 64.0, 44.0, 48.0, 43.0",0


In [13]:
test.head()

,rowid,unique_id,types,abilities,base_happiness,height_m,weight_kg,poke_stats
0,1,602,"grass, nan","['Overgrow', 'Contrary']",70.0,0.6,8.1,"nan, nan, 55.0, 45.0, nan, 63.0"
1,2,756,"fairy, flying","['Hustle', 'Serene Grace', 'Super Luck']",70.0,1.5,38.0,"nan, nan, 95.0, 120.0, 115.0, 80.0"
2,3,191,"electric, nan","['Static', 'Plus']",70.0,1.4,61.5,"nan, 95.0, 105.0, 165.0, nan, 45.0"
3,4,16,"psychic, nan",['Levitate'],None,0.2,0.6,"nan, 30.0, 50.0, 65.0, nan, 45.0"
4,5,718,"fighting, psychic","['Pure Power', 'Telepathy']",None,1.3,31.5,"nan, 100.0, 85.0, 80.0, 85.0, 100.0"


In [14]:
print(len(train),len(test))

520 281


What to do: 
separate abilities
plot things out-happiness v. legendary, weight v. legendary, height v. legendary, percentage of types legendary, percentage of abiliies legendary,
create a single type category, create number of abilities, abilities categorized? probably not, height to weight ratio(BMI)
find and fill null values

In [15]:
types=train['types']
what=types[4].split(',')
what[1]

' nan'

How to go about it

In [16]:
train['type1']=train['types'].apply(lambda x: x.split(',')[0])
train['type2']=train['types'].apply(lambda x: x.split(',')[1])
train['hp']=train['poke_stats'].apply(lambda x: x.split(',')[0])
train['attack']=train['poke_stats'].apply(lambda x: x.split(',')[1])
train['defense']=train['poke_stats'].apply(lambda x: x.split(',')[2])
train['sp. attack']=train['poke_stats'].apply(lambda x: x.split(',')[3])
train['sp. defense']=train['poke_stats'].apply(lambda x: x.split(',')[4])
train['speed']=train['poke_stats'].apply(lambda x: x.split(',')[5])
test['type1']=test['types'].apply(lambda x: x.split(',')[0])
test['type2']=test['types'].apply(lambda x: x.split(',')[1])
test['hp']=test['poke_stats'].apply(lambda x: x.split(',')[0])
test['attack']=test['poke_stats'].apply(lambda x: x.split(',')[1])
test['defense']=test['poke_stats'].apply(lambda x: x.split(',')[2])
test['sp. attack']=test['poke_stats'].apply(lambda x: x.split(',')[3])
test['sp. defense']=test['poke_stats'].apply(lambda x: x.split(',')[4])
test['speed']=test['poke_stats'].apply(lambda x: x.split(',')[5])

In [17]:
train=train.drop(columns=['types','poke_stats'])
test=test.drop(columns=['types','poke_stats'])

In [18]:
test.isna().any()

rowid             False
unique_id         False
abilities         False
base_happiness     True
height_m           True
weight_kg          True
type1             False
type2             False
hp                False
attack            False
defense           False
sp. attack        False
sp. defense       False
speed             False
dtype: bool

In [19]:
train.hp.unique()

array(['nan', '109', '74', '43', '144', '65', '46', '60', '50', '70',
       '100', '106', '35', '58', '75', '45', '48', '79', '80', '66', '55',
       '83', '64', '40', '85', '137', '42', '110', '38', '115', '30',
       '51'], dtype=object)

In [20]:
train

,rowid,unique_id,abilities,base_happiness,height_m,weight_kg,is_legendary,type1,type2,hp,attack,defense,sp. attack,sp. defense,speed
0,1,665,"['Guts', 'Sheer Force', 'Iron Fist']",70.0,0.6,12.5,0,fighting,nan,nan,80.0,55.0,25.0,35.0,35.0
1,2,288,"['Bulletproof', 'Soundproof', 'Overcoat']",70.0,0.6,29.7,0,dragon,nan,nan,nan,65.0,45.0,45.0,45.0
2,3,349,"['Corrosion', 'Oblivious']",70.0,1.2,22.2,0,poison,fire,nan,64.0,60.0,111.0,60.0,117.0
3,4,260,"['Rock Head', 'Lightningrod', 'Battle Armor']",70.0,0.4,6.5,0,ground,nan,nan,50.0,95.0,40.0,50.0,35.0
4,5,536,"['Torrent', 'Sheer Force']",70.0,0.6,9.5,0,water,nan,nan,65.0,64.0,44.0,48.0,43.0
5,6,293,['Turboblaze'],0.0,3.2,330.0,1,dragon,fire,nan,120.0,100.0,150.0,120.0,90.0
6,7,181,['Levitate'],70.0,1.1,15.3,0,ground,dragon,nan,70.0,50.0,50.0,50.0,70.0
7,8,554,"['Poison Point', 'Rivalry', 'Sheer Force']",70.0,1.3,60.0,0,poison,ground,nan,92.0,87.0,75.0,85.0,76.0
8,9,507,['Battery'],70.0,0.5,10.5,0,bug,electric,nan,82.0,95.0,55.0,75.0,36.0
9,10,140,['Beast Boost'],0.0,1.2,55.5,1,rock,poison,109,53.0,47.0,127.0,131.0,103.0


In [21]:
type1_count=train.groupby('type1')['rowid'].nunique()
type2_count=train.groupby('type2')['rowid'].nunique()
print(type1_count)
print(type2_count)

type1
bug         39
dark        20
dragon      20
electric    28
fairy       13
fighting    15
fire        38
flying       1
ghost       17
grass       50
ground      18
ice         16
normal      65
poison      24
psychic     36
rock        29
steel       15
water       76
Name: rowid, dtype: int64
type2
 bug           4
 dark         17
 dragon       11
 electric      5
 fairy        17
 fighting     18
 fire          7
 flying       60
 ghost        11
 grass        13
 ground       26
 ice           7
 nan         256
 normal        2
 poison       26
 psychic      13
 rock          7
 steel        12
 water         8
Name: rowid, dtype: int64


In [22]:
legendary=train[train['is_legendary']=='1']

In [23]:
legendary

,rowid,unique_id,abilities,base_happiness,height_m,weight_kg,is_legendary,type1,type2,hp,attack,defense,sp. attack,sp. defense,speed
5,6,293,['Turboblaze'],0.0,3.2,330.0,1,dragon,fire,nan,120.0,100.0,150.0,120.0,90.0
9,10,140,['Beast Boost'],0.0,1.2,55.5,1,rock,poison,109,53.0,47.0,127.0,131.0,103.0
38,39,236,['Natural Cure'],100.0,0.6,5.0,1,psychic,grass,nan,nan,100.0,100.0,nan,100.0
46,47,640,['Bad Dreams'],0.0,1.5,50.5,1,dark,nan,nan,90.0,90.0,135.0,90.0,125.0
51,52,563,['Beast Boost'],0.0,3.8,100.0,1,electric,nan,nan,nan,71.0,nan,71.0,83.0
61,62,66,"['Clear Body', 'Ice Body']",None,1.8,175.0,1,ice,nan,nan,nan,nan,100.0,200.0,50.0
69,70,634,['Drizzle'],0.0,4.5,352.0,1,water,nan,nan,150.0,90.0,180.0,160.0,90.0
82,83,102,['Serene Grace'],100.0,0.3,1.1,1,steel,psychic,100,100.0,100.0,100.0,100.0,100.0
101,102,42,"['Pressure', 'Regenerator']",None,3.8,199.0,1,fire,flying,106,nan,90.0,110.0,nan,90.0
109,110,458,['Prism Armor'],0.0,2.4,230.0,1,psychic,nan,nan,nan,101.0,127.0,nan,79.0


In [24]:
len(legendary)

48

In [25]:
for i in range(15):
    tnull=train[train.columns[i]].isnull().sum()
    if tnull>0:
        print(train.columns[i],":",tnull)

base_happiness : 47
height_m : 14
weight_kg : 14


In [26]:
train.base_happiness.unique()

array(['70.0', '0.0', '35.0', '100.0', None, '140.0', '90.0'],
      dtype=object)

In [27]:
sadmon=train[train['base_happiness']=='0.0']
sadmon

,rowid,unique_id,abilities,base_happiness,height_m,weight_kg,is_legendary,type1,type2,hp,attack,defense,sp. attack,sp. defense,speed
5,6,293,['Turboblaze'],0.0,3.2,330.0,1,dragon,fire,nan,120.0,100.0,150.0,120.0,90.0
9,10,140,['Beast Boost'],0.0,1.2,55.5,1,rock,poison,109,53.0,47.0,127.0,131.0,103.0
46,47,640,['Bad Dreams'],0.0,1.5,50.5,1,dark,nan,nan,90.0,90.0,135.0,90.0,125.0
51,52,563,['Beast Boost'],0.0,3.8,100.0,1,electric,nan,nan,nan,71.0,nan,71.0,83.0
69,70,634,['Drizzle'],0.0,4.5,352.0,1,water,nan,nan,150.0,90.0,180.0,160.0,90.0
106,107,279,"['Run Away', 'Klutz', 'Limber']",0.0,0.4,5.5,0,normal,nan,nan,66.0,44.0,44.0,56.0,85.0
109,110,458,['Prism Armor'],0.0,2.4,230.0,1,psychic,nan,nan,nan,101.0,127.0,nan,79.0
132,133,388,['Dark Aura'],0.0,5.8,203.0,1,dark,flying,nan,nan,95.0,nan,98.0,99.0
136,137,773,['Pressure'],0.0,1.7,60.8,1,psychic,nan,nan,95.0,90.0,nan,90.0,180.0
164,165,598,['Soul-Heart'],0.0,1.0,80.5,1,steel,fairy,nan,nan,115.0,130.0,115.0,65.0


In [28]:
len(sadmon)

24

In [29]:
unknown=train[train['base_happiness'].isnull()]
unknown

,rowid,unique_id,abilities,base_happiness,height_m,weight_kg,is_legendary,type1,type2,hp,attack,defense,sp. attack,sp. defense,speed
36,37,71,"['Sand Veil', 'Sand Rush', 'Snow Cloak', 'Slus...",None,None,None,0,ground,ice,nan,75.0,90.0,10.0,35.0,40.0
49,50,738,['Shed Skin'],None,0.7,11.5,0,bug,nan,nan,35.0,55.0,25.0,nan,15.0
61,62,66,"['Clear Body', 'Ice Body']",None,1.8,175.0,1,ice,nan,nan,nan,nan,100.0,200.0,50.0
70,71,757,"['Prankster', 'Infiltrator', 'Chlorophyll']",None,0.3,0.6,0,grass,fairy,nan,27.0,60.0,37.0,nan,66.0
92,93,605,"['Chlorophyll', 'Leaf Guard', 'Infiltrator']",None,0.8,3.0,0,grass,flying,nan,nan,70.0,55.0,95.0,110.0
97,98,64,"['Soundproof', 'Rattled']",None,0.6,16.3,0,normal,nan,nan,51.0,23.0,51.0,nan,28.0
101,102,42,"['Pressure', 'Regenerator']",None,3.8,199.0,1,fire,flying,106,nan,90.0,110.0,nan,90.0
104,105,592,"['Limber', 'Unburden', 'Mold Breaker']",None,0.8,21.5,0,fighting,flying,nan,92.0,75.0,74.0,63.0,118.0
116,117,207,"['Synchronize', 'Inner Focus', 'Magic Guard']",None,0.9,19.5,0,psychic,nan,nan,20.0,15.0,105.0,55.0,90.0
131,132,405,"['Run Away', 'Early Bird', 'Tangled Feet']",None,1.8,85.2,0,normal,flying,nan,110.0,70.0,60.0,60.0,110.0
